In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
import pickle
from string import punctuation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

!pip install pymystem3
from pymystem3 import Mystem

nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
recipes=pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv',
                   converters={'tags': lambda x: x[1:-1].replace('"', '').replace("'", "").replace(" ", "").split(',')}
                   )

recipes["Text"] = recipes["name"] + ' ' + recipes["description"] + ' ' + recipes["steps"]
recipes = recipes.drop(['id', 'minutes','contributor_id', 'nutrition', 'n_steps', 'n_ingredients', 'submitted', 'name', 'description', 'steps', 'ingredients'], axis=1)
recipes.head()

In [ ]:
stop_list = ['ThrowtheultimatefiestawiththissopaipillasrecipefromFood.com.', 
               'preparation', 'time-to-make', 'course', 'cuisine', 'low-in-something',
               'equipment', 'number-of-servings', 'healthy-2', 'free-of-something' ,'technique',
               'high-in-something', 'high-in-something-diabetic-friendly', 
               'less_thansql:name_topics_of_recipegreater_than', '15-minutes-or-less', '30-minutes-or-less', '60-minutes-or-less', 
               '4-hours-or-less', '3-steps-or-less', '5-ingredients-or-less', 'very-low-carbs', '1-day-or-more', 'copycat',
               'oamc-freezer-make-ahead', 'heirloom-historical', 'main-ingredient', 'taste-mood', 'low-sodium', 'low-carb',
               'low-cholesterol','low-protein', 'low-saturated-fat', 'comfort-food', 'presentation', 'low-fat', 'condiments-etc',
               'high-protein', 'southern-united-states', 'gifts', 'high-calcium', 'novelty', 'south-west-pacific', 
               'southwestern-united-states', 'brown-bag', 'midwestern', 'northeastern-united-states', 'independence-day',
               'superbowl', 'wild-game', 'crusts-pastry-dough-2', 'high-fiber', 'super-bowl', 'amish-mennonite', 'soul',
               'jewish-sephardi', 'pennsylvania-dutch', 'hidden-valley-ranch', 'reynolds-wrap', 'memorial-day', 'labor-day',
               'mothers-day', 'unprocessed-freezer', 'ragu-recipes-contest', 'simply-potatoes2', 'heirloom-historical-recipes',
               'weeknight', 'no-cook', 'potluck', 'collard-greens', 'one-dish-meal', 'small-appliance', 'cooking-mixes',
               'gluten-free', 'californian', 'diabetic', 'finger-food', 'tempeh', 'pacific-northwest', 'british-columbian',
             'kwanzaa', 'from-scratch','gumbo','cajun','creole','orange-roughy','leftovers','egg-free',
             'pitted-fruit','fudge','celebrity','baja','dairy-free','flat-shapes','cinco-de-mayo','rosh-hashana',
             'chutneys','mardi-gras-carnival','bok-choys','water-bath','stocks','birthday','nut-free',
             'college','fathers-day','dehydrator','april-fools-day','for-large-groups-holiday-event'
              ]

recipes['tags'] = recipes['tags'].apply(
    lambda row: [val for val in row if val not in stop_list]
)

In [ ]:
def clean_text(text):
    text = re.sub("\'", "", text)
    text = re.sub("," , " ", text)
    text = re.sub("[^a-zA-Z]"," ",text) 
    text = ' '.join(text.split()) 
    text = text.lower() 
    return text

recipes.Text = recipes.Text.fillna(" no recipe ")
recipes.Text = recipes.Text.apply(lambda x: clean_text(x))

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

recipes.Text = recipes.Text.apply(lambda x: remove_stopwords(x))

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    lemmatized_text=[]
    sentence_words = nltk.word_tokenize(text)
    for word in sentence_words:
        lemmatized_text.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    lemmatized_text = ' '.join(lemmatized_text)
    return lemmatized_text

recipes.Text = recipes.Text.apply(lambda x: lemmatize_text(x))

def translate(text):
    dictionary = {'dietary': 'диетическое', 
                  'easy': 'легко готовить', 
                  'occasion': 'к особому случаю', 
                  'main-dish': 'главное блюдо', 
                  'chili':'соуч-чили', 
                  'crock-pot-slow-cooker':'на медленном огне',
                  'north-american':'североамериканская',
                  'side-dishes':'гарниры',
                  'vegetables':'овощи',
                  'mexican':'мексиканская',
                  'fall':'осеннее',
                  'holiday-event':'к празднику',
                  'vegetarian':'вегетарианская',
                  'winter':'зимнее',
                  'christmas':'Рождество и Новый Год',
                  'seasonal':'сезонное',
                  'squash':'кабачки',
                  'breakfast':'завтрак',
                  'pork':'свинина',
                  'american':'североамериканская',
                  'oven':'для духовки',
                  'kid-friendly':'для детей',
                  'pizza':'пицца',
                  'meat':'мясо',
                  'eggs-dairy':'яйце-молочное',
                  'potatoes':'картофель',
                  'dinner-party':'званый ужин',
                  'easter':'Пасха',
                  'cheese':'сыр',
                  'stove-top':'плита',
                  'new-years':'Рождество и Новый Год',
                  'thanksgiving':'День Благодарения',
                  'st-patricks-day':'День Св. Патрика',
                  'valentines-day':'День Св. Валентина',
                  'inexpensive':'недорогое',
                  'brunch':'поздний завтрак',
                  'served-hot':'подавать горячим',
                  'canning':'консервирование',
                  'beverages':'напитки',
                  'fruit':'фрукты',
                  'shakes':'шейки',
                  'low-calorie':'низкокалорийное',
                  'apples':'яблоки',
                  'served-cold':'подавать холодным',
                  'appetizers':'закуски',
                  'canadian':'североамериканская',
                  'vegan':'веганское',
                  'freezer':'морозильник',
                  'hawaiian':'гавайская',
                  'spicy':'острое',
                  'pork-ribs':'свиные ребрышки',
                  'savory':'не сладкое',
                  'sweet':'сладкое',
                  'pies-and-tarts':'пирожки и пироги',
                  'desserts':'десерты',
                  'lunch':'обед',
                  'snacks':'закуски',
                  'refrigerator':'холодильник',
                  'frozen-desserts':'замороженные десерты',
                  'pies':'пирожки и пироги',
                  'chocolate':'шоколад',
                  'breads':'хлеб',
                  'quick-breads':'хлеб',
                  'tropical-fruit':'тропические фрукты',
                  'bananas':'бананы',
                  'healthy':'здоровая еда',
                  'berries':'ягоды',
                  'strawberries':'клубника',
                  'beans':'бобы',
                  'to-go':'в дорогу',
                  'greens':'зелень',
                  'pork-chops':'свиные отбивные',
                  'deep-fry':'жарка во фритюре',
                  'candy':'конфеты',
                  'beef':'говядина',
                  'lactose':'лактоза',
                  'bisques-cream-soups':'супы',
                  'soups-stews':'супы',
                  'spring':'весеннее',
                  'food-processor-blender':'для блендера',
                  'cauliflower':'цветная капуста',
                  'beginner-cook':'просто готовить',
                  'spinach':'шпинат',
                  'sauces':'соусы',
                  'eggs':'яйца',
                  'savory-sauces':'несладкие соусы',
                  'asian':'азиатская',
                  'indian':'индийская',
                  'herb-and-spice-mixes':'травы и специи',
                  'peppers':'перец',
                  'poultry':'птица',
                  'chicken':'курица',
                  'black-beans':'черные бобы',
                  'grilling':'для гриля',
                  'picnic':'для пикника',
                  'cookies-and-brownies':'печенье',
                  'bar-cookies':'печенье',
                  'brownies':'брауни',
                  'casseroles':'запеканки',
                  'ground-beef':'фарш',
                  'deer':'олень',
                  'summer':'летняя',
                  'toddler-friendly':'для малышей',
                  'mixer':'миксер',
                  'clear-soups':'супы',
                  'salads':'салаты',
                  'pasta':'паста',
                  'pasta-rice-and-grains':'паста, рис и крупы',
                  'for-large-groups':'для больших групп',
                  'muffins':'маффины',
                  'sandwiches':'сендвичи',
                  'cakes':'торты',
                  'nuts':'орехи',
                  'pineapple':'ананас',
                  'corn':'кукуруза',
                  'german':'немецкая',
                  'european':'европейская',
                  'polish':'польская',
                  'russian':'русская',
                  'yeast':'дрожжи',
                  'italian':'итальянская',
                  'lasagna':'лазанья',
                  'omelets-and-frittatas':'омлеты',
                  'for-1-or-2':'для одного или двоих',
                  'romantic':'романтическая',
                  'ramadan':'Рамадан',
                  'chicken-breasts':'куриные грудки',
                  'dips':'соусы',
                  'barbecue':'барбекю',
                  'marinades-and-rubs':'маринады',
                  'infant-baby-friendly':'для малышей',
                  'roast':'жареное',
                  'veal':'телятина',
                  'garnishes':'гарниры',
                  'cheesecake':'чизкейк',
                  'grains':'крупы',
                  'wedding':'на свадьбу',
                  'kosher':'кошерная',
                  'mushrooms':'грибы',
                  'seafood':'морепродукты',
                  'fish':'рыба',
                  'tuna':'тунец',
                  'saltwater-fish':'рыба',
                  'elbow-macaroni':'макароны',
                  'pancakes-and-waffles':'блины и вафли',
                  'chicken-thighs-legs':'куриные ножки',
                  'salmon':'лосось',
                  'indonesian':'индонезийская',
                  'soy-tofu':'тофу',
                  'spreads':'спреды',
                  'beef-ribs':'говяжьи ребрышки',
                  'grapes':'виноград',
                  'smoothies':'смузи',
                  'bacon':'бекон',
                  'broil':'жареное',
                  'tomatoes':'помидоры',
                  'cocktails':'коктейли',
                  'no-shell-fish':'рыба',
                  'squid':'морепродукты',
                  'chinese':'китайская',
                  'steam':'на пару',
                  'onions':'лук',
                  'broccoli':'брокколи',
                  'meatloaf':'мясной рулет',
                  'rolled-cookies':'печенье',
                  'drop-cookies':'печенье',
                  'cake-fillings-and-frostings':'заправка для тортов',
                  'lettuces':'салат',
                  'stews':'рагу',
                  'lentils':'чечевица',
                  'danish':'датская',
                  'scandinavian':'скандинавская',
                  'swiss':'швейцарская',
                  'swedish':'шведская',
                  'french':'французская',
                  'microwave':'микроволновка',
                  'granola-and-porridge':'мюсли и каши',
                  'bread-machine':'хлеб',
                  'asparagus':'спаржа',
                  'african':'африканская',
                  'shrimp':'креветки',
                  'shellfish':'моллюски',
                  'savory-pies':'пироги',
                  'salad-dressings':'заправка для салата',
                  'simply-potatoes':'картофель',
                  'crab':'краб',
                  'mango':'манго',
                  'curries':'карри',
                  'roast-beef':'жареная говядина',
                  'pork-sausage':'свиные сосиски',
                  'turkey':'индейка',
                  'whole-turkey':'индейка',
                  'green-yellow-beans':'бобы',
                  'beef-sausage':'говяжьи сосиски',
                  'australian':'австралийская',
                  'rice':'рис',
                  'brown-rice':'рис',
                  'white-rice':'рис',
                  'long-grain-rice':'рис',
                  'caribbean':'карибская',
                  'central-american':'центральноамериканская',
                  'stuffings-dressings':'заправки',
                  'english':'английская',
                  'spaghetti':'спагетти',
                  'coconut':'кокос',
                  'quebec':'квебекская',
                  'freshwater-fish':'рыба',
                  'catfish':'сом',
                  'middle-eastern':'ближневосточная',
                  'lebanese':'ливанская',
                  'ravioli-tortellini':'равиоли',
                  'steak':'равиоли',
                  'clams':'моллюски',
                  'rolls-biscuits':'печенье',
                  'gelatin':'желатин',
                  'greek':'греческая',
                  'citrus':'цитрусовые',
                  'oranges':'апельсин',
                  'papaya':'папайя',
                  'punch':'пунш',
                  'peaches':'персики',
                  'moose':'лось',
                  'wings':'крылышки',
                  'pasta-shells':'макароны-ракушки',
                  'blueberries':'черника',
                  'south-american':'южноамериканская',
                  'sweet-sauces':'сладкие соусы',
                  'pork-loins':'свиное филе',
                  'ham':'ветчина',
                  'burgers':'бургеры',
                  'biscotti':'бисквит',
                  'peanut-butter':'арахисовое масло',
                  'halloween':'Хэллоуин',
                  'zucchini':'цуккини',
                  'puddings-and-mousses':'пуддинги',
                  'japanese':'японская',
                  'lemon':'лимон',
                  'chick-peas-garbanzos':'нут',
                  'cobblers-and-crisps':'чипсы',
                  'lamb-sheep':'баранина',
                  'turkey-burgers':'бургеры из индейки',
                  'carrots':'морковь',
                  'penne':'пенне',
                  'micro-melanesia':'океанская',
                  'yams-sweet-potatoes':'картофель',
                  'native-american':'индейская',
                  'artichoke':'артишок',
                  'salsas':'сальса',
                  'cupcakes':'капкейки',
                  'puerto-rican':'центральноамериканская',
                  'stir-fry':'жарить в масле',
                  'hand-formed-cookies':'печенье',
                  'spanish':'испанская',
                  'steaks':'стейк',
                  'irish':'ирландская',
                  'scones':'булки',
                  'oaxacan':'центральноамериканская',
                  'lime':'лайм',
                  'camping':'на свежем воздхе',
                  'thai':'тайская',
                  'polynesian':'полинезийская',
                  'melons':'дыня',
                  'jewish-ashkenazi':'еврейская',
                  'hanukkah':'Ханука',
                  'iraqi':'иракская',
                  'saudi-arabian':'арабская',
                  'whole-chicken':'цыпленок',
                  'sourdough':'закваска',
                  'baking':'выпечка',
                  'medium-grain-rice':'рис',
                  'pakistani':'пакистанская',
                  'scottish':'шотландская',
                  'somalian':'сомалийская',
                  'non-alcoholic':'безалкогольный',
                  'ontario':'североамериканская',
                  'raspberries':'малина',
                  'coffee-cakes':'торты',
                  'tex-mex':'мексиканская',
                  'south-african':'южноафриканская',
                  'passover':'Пасха',
                  'new-zealand':'новозеландская',
                  'chowders':'супы',
                  'pressure-cooker':'скороварка',
                  'whitefish':'рыба',
                  'halibut':'палтус',
                  'ice-cream':'мороженое',
                  'pears':'груша',
                  'scallops':'гребешки',
                  'pheasant':'фазан',
                  'tarts':'пироги',
                  'chinese-new-year':'китайская',
                  'chicken-livers':'куриная печень',
                  'manicotti':'паста',
                  'szechuan':'китайская',
                  'fillings-and-frostings-chocolate':'глазурь',
                  'jams-and-preserves':'варенье',
                  'cherries':'вишня',
                  'chard':'зелень',
                  'tilapia':'рыба',
                  'colombian':'колумбийская',
                  'welsh':'валлийская',
                  'short-grain-rice':'рис',
                  'kiwifruit':'киви',
                  'breakfast-eggs':'яйца',
                  'eggs-breakfast':'яйца',
                  'czech':'чешская',
                  'filipino':'филиппинская',
                  'turkey-breasts':'индейка',
                  'cuban':'кубинская',
                  'perch':'окунь',
                  'smoker':'копченое',
                  'belgian':'бельгиская',
                  'costa-rican':'центральноамериканская',
                  'guatemalan':'центральноамериканская',
                  'honduran':'южноамериканская',
                  'finnish':'финская',
                  'moroccan':'морокканская',
                  'a1-sauce':'соус',
                  'avocado':'авокадо',
                  'jellies':'желе',
                  'iranian-persian':'иранская',
                  'dutch':'голландская',
                  'pumpkin':'тыква',
                  'cod':'треска',
                  'sole-and-flounder':'камбала',
                  'plums':'сливы',
                  'turkish':'турецкая',
                  'portuguese':'португальская',
                  'hungarian':'венгерская',
                  'georgian':'грузинская',
                  'brazilian':'бразильская',
                  'nigerian':'нигерийская',
                  'ethiopian':'эфиопская',
                  'sudanese':'суданская',
                  'congolese':'конголезская',
                  'ecuadorean':'центральноамериканская',
                  'mussels':'моллюски',
                  'peruvian':'перуанская',
                  'meatballs':'тефтели',
                  'egyptian':'египетская',
                  'argentine':'аргентинская',
                  'mahi-mahi':'рыба',
                  'chilean':'чилийская',
                  'crawfish':'раки',
                  'oatmeal':'овсянка',
                  'malaysian':'малазийская',
                  'duck':'утка',
                  'rosh-hashanah':'еврейская',
                  'nepalese':'непальская',
                  'eggplant':'баклажан',
                  'goose':'гусь',
                  'icelandic':'исландская',
                  'veggie-burgers':'вегетарианская',
                  'mashed-potatoes':'картофельное пюре',
                  'chocolate-chip-cookies':'печенье',
                  'vietnamese':'вьетнамская',
                  'elk':'лось',
                  'oysters':'устрицы',
                  'palestinian':'палестинская',
                  'rabbit':'кролик',
                  'norwegian':'норвежская',
                  'lobster':'омар',
                  'main-dish-beef':'говядина',
                  'beef-crock-pot':'говядина',
                  'pot-roast':'тушеное',
                  'hunan':'китайская',
                  'austrian':'австрийская',
                  'halloween-cocktails':'коктейли',
                  'libyan':'ливийская',
                  'beef-organ-meats':'говядина',
                  'beef-liver':'говяжья печень',
                  'angolan':'ангольская',
                  'korean':'корейская',
                  'cantonese':'китайская',
                  'whole-duck':'утка',
                  'halloween-cupcakes':'капкейки',
                  'halloween-cakes':'торты',
                  'ragu-recipe-contest':'рагу',
                  'macaroni-and-cheese':'паста',
                  'trout':'форель',
                  'irish-st-patricks-day':'День Св. Патрика',
                  'bass':'окунь',
                  'beijing':'китайская',
                  'quail':'перепел',
                  'pickeral':'щука',
                  'main-dish-chicken':'курица',
                  'bear':'медведь',
                  'brewing':'пивоварение',
                  'cambodian':'камбоджийская',
                  'beef-kidney':'говядина',
                  'beef-sauces':'говядина',
                  'spaghetti-sauce':'соусы к спагетти',
                  'pot-pie':'пирог',
                  'dips-lunch-snacks':'закуски',
                  'pressure-canning':'консервы',
                  'main-dish-pork':'свинина',
                  'pork-loins-roast':'свинина',
                  'quiche':'пирог',
                  'sugar-cookies':'печенье',
                  'venezuelan':'венесуэльская',
                  'octopus':'осьминог',
                  'side-dishes-beans':'фасоль',
                  'beans-side-dishes':'фасоль',
                  'baked-beans':'фасоль',
                  'snacks-kid-friendly':'закуски',
                  'snacks-sweet':'закуски',
                  'duck-breasts':'утка',
                  'main-dish-pasta':'паста',
                  'pasta-salad':'паста',
                  'pasta-elbow-macaroni':'паста',
                  'namibian':'африканская',
                  'mushroom-soup':'грибной суп',
                  'beef-barley-soup':'супы',
                  'pork-crock-pot':'свинина',
                  'crock-pot-main-dish':'глиняный горшок',
                  'desserts-fruit':'десерты',
                  'desserts-easy':'десерты',
                  'lasagne':'лазанья',
                  'laotian':'лаосская',
                  'cabbage':'капуста',
                  'prepared-potatoes':'картофель',
                  'middle-eastern-main-dish':'ближневосточная',
                  'dips-summer':'летнее',
                  'black-bean-soup':'супы',
                  'bean-soup':'супы',
                  'served-hot-new-years':'Новый Год',
                  'ham-and-bean-soup':'супы',
                  'lamb-sheep-main-dish':'баранина',
                  'mongolian':'монгольская',
                  'stews-poultry':'рагу',
                  'pasta-rice-and-grains-elbow-macaroni':'паста',
                  'pork-loin':'свинина',
                  'cranberry-sauce':'клюквенный соус',
                  'marinara-sauce':'соус Маринара',
                  'breakfast-casseroles':'запеканка',
                  'breakfast-potatoes':'картофель',
                  'main-dish-seafood':'рыба',
                  'shrimp-main-dish':'креведки',
                  'pumpkin-bread':'тыква',
                  'bread-pudding':'хлеб',
                  'chicken-crock-pot':'курица',
                  'chicken-stews':'курица',
                  'chicken-stew':'курица',
                  'roast-beef-main-dish':'говядина',
                  'roast-beef-comfort-food':'говядина'
                  '':''
                  
                  
                 } 
    c=[]
    for tag in text:
        c.append(tag.replace(tag, dictionary[tag]))
    return c

recipes.tags = recipes.tags.apply(lambda x: translate(x))

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(recipes['tags'])

y = multilabel_binarizer.transform(recipes['tags'])

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

xtrain, xval, ytrain, yval = train_test_split(recipes.Text, y, test_size=0.2, random_state=9)
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

lr = LogisticRegression(max_iter = 1000, class_weight='balanced')
clf = OneVsRestClassifier(lr)
clf.fit(xtrain_tfidf, ytrain)

In [ ]:
y_pred = clf.predict(xval_tfidf)
f1_score(yval, y_pred, average="micro")

In [ ]:
def infer_tags(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q = lemmatize_text(q)
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

infer_tags('fried chicken')

In [ ]:
model_filename = 'multilabel_model.sav'
pickle.dump(clf, open(model_filename, 'wb'))

tfidf = 'tfidf.sav'
pickle.dump(tfidf_vectorizer, open(tfidf, 'wb'))

model_filename = 'multilabel_binarizer.sav'
pickle.dump(multilabel_binarizer, open(model_filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open('/kaggle/working/multilabel_model.sav', 'rb'))
tfidf_vect_loaded = pickle.load(open('/kaggle/working/tfidf.sav', 'rb'))
binarizer = pickle.load(open('/kaggle/working/multilabel_binarizer.sav', 'rb'))

def infer_tags2(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q = lemmatize_text(q)
    q_vec = tfidf_vect_loaded.transform([q])
    q_pred = loaded_model.predict(q_vec)
    return binarizer.inverse_transform(q_pred)
infer_tags2(" pizza ")